In [75]:
import os

In [76]:
%pwd

'd:\\Runway\\Projects\\data_it_enterprises'

In [ ]:
os.chdir("../")

In [77]:
%pwd

'd:\\Runway\\Projects\\data_it_enterprises'

In [85]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: float

In [86]:
from data_it_entp.constants import *
from data_it_entp.utils.common import read_yaml,create_directories

In [87]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [88]:
import pandas as pd
import os
from data_it_entp import logger
from sklearn.linear_model import ElasticNet
import joblib

In [89]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        features = ['CPU_Load', 'Memory_Usage', 'Disk_Usage']


        train_x = train_data[features]
        print(f"train x={train_x}")
        test_x = test_data[features]
        print(f"test x={test_x}")
        train_y = train_data[[self.config.target_column]]
        print(train_y)
        test_y = test_data[[self.config.target_column]]
        print(test_y)


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [90]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-11-08 18:25:18,777:INFO:common:yaml file: config\config.yaml loaded successfully:]
[2023-11-08 18:25:18,780:INFO:common:yaml file: params.yaml loaded successfully:]
[2023-11-08 18:25:18,780:INFO:common:yaml file: schema.yaml loaded successfully:]
[2023-11-08 18:25:18,785:INFO:common:created directory at: artifacts:]
[2023-11-08 18:25:18,785:INFO:common:created directory at: artifacts/model_trainer:]
train x=       CPU_Load  Memory_Usage  Disk_Usage
0     46.695877     48.236401   56.738613
1     65.626312     51.409185   49.332597
2     52.075837     62.546858   32.986407
3     50.119382     56.777095   38.856738
4     27.700496     57.304656   43.837445
...         ...           ...         ...
3745  51.675067     48.709561   52.419553
3746  70.333609     50.694207   42.020465
3747  32.554001     55.293972   48.544236
3748  54.243945     44.477510   22.788593
3749  37.938911     38.512457   41.401396

[3750 rows x 3 columns]
test x=       CPU_Load  Memory_Usage  Disk_Usage
0    